# Advent of Code 2023
## Day 5
*<https://adventofcode.com/2023/day/5>*

In [1]:
import heapq
import math
import re
from collections import Counter, defaultdict, deque, namedtuple
from itertools import combinations, permutations, product
from string import ascii_letters, ascii_lowercase, ascii_uppercase

import IPython
import z3
from rich import inspect, pretty, print

from new_helper import *

pretty.install()

In [2]:
DAY = 5
input_str = get_aoc_input(DAY, 2023)
part_1 = part_2 = 0

In [3]:
inp = input_str.parse_groups()

In [4]:
seeds = ints(inp[0][0])
maps = defaultdict(list[tuple[int, int, int]])
proc = []
for mp in inp[1:]:
    name = mp[0].split()[0]
    source = name.split("-")[0]
    dest = name.split("-")[2]
    proc.append((source, dest))

    for line in mp[1:]:
        dest_start, source_start, range_len = ints(line)
        maps[(source, dest)].append((source_start, dest_start, range_len))

In [5]:
def convert(source, dest, val):
    for source_start, dest_start, range_len in maps[(source, dest)]:
        if source_start <= val < source_start + range_len:
            return dest_start + val - source_start

    return val

In [6]:
part_1 = float("inf")
for seed in seeds:
    soil = convert("seed", "soil", seed)
    fertilizer = convert("soil", "fertilizer", soil)
    water = convert("fertilizer", "water", fertilizer)
    light = convert("water", "light", water)
    temperature = convert("light", "temperature", light)
    humidity = convert("temperature", "humidity", temperature)
    location = convert("humidity", "location", humidity)

    part_1 = min(part_1, location)

In [7]:
def convert_ranges(rs: set[range], ms: dict[range, int]):
    new = set()

    while rs:
        r = rs.pop()
        for m, off in ms.items():
            if m.start < r.start < r.stop < m.stop:
                new.add(range(r.start + off, r.stop + off))
            elif r.start < m.start < m.stop < r.stop:
                rs.add(range(r.start, m.start))
                rs.add(range(m.stop, r.stop))
                new.add(range(m.start + off, m.stop + off))
            elif r.start < m.start < r.stop < m.stop:
                rs.add(range(r.start, m.start))
                new.add(range(m.start + off, r.stop + off))
            elif m.start < r.start < m.stop < r.stop:
                rs.add(range(m.stop, r.stop))
                new.add(range(r.start + off, m.stop + off))
            elif m.start == r.start < m.stop == r.stop:
                new.add(range(m.start + off, m.stop + off))
            elif m.start == r.start < r.stop < m.stop:
                new.add(range(m.start + off, r.stop + off))
            elif m.start == r.start < m.stop < r.stop:
                rs.add(range(m.stop, r.stop))
                new.add(range(m.start + off, m.stop + off))
            elif m.start < r.start < m.stop == r.stop:
                new.add(range(r.start + off, m.stop + off))
            elif r.start < m.start < r.stop == m.stop:
                rs.add(range(r.start, m.start))
                new.add(range(m.start + off, m.stop + off))
            else:
                continue
            break

        else:
            new.add(r)

    return new

In [8]:
def convert_to_ms(maps: list[tuple[int, int, int]]):
    return {range(source, source + offset): dest - source for source, dest, offset in maps}

In [9]:
part_2 = float("inf")
for i in range(0, len(seeds), 2):
    r = range(seeds[i], seeds[i] + seeds[i + 1])
    soil = convert_ranges({r}, convert_to_ms(maps[("seed", "soil")]))
    fertilizer = convert_ranges(soil, convert_to_ms(maps[("soil", "fertilizer")]))
    water = convert_ranges(fertilizer, convert_to_ms(maps[("fertilizer", "water")]))
    light = convert_ranges(water, convert_to_ms(maps[("water", "light")]))
    temperature = convert_ranges(light, convert_to_ms(maps[("light", "temperature")]))
    humidity = convert_ranges(temperature, convert_to_ms(maps[("temperature", "humidity")]))
    location = convert_ranges(humidity, convert_to_ms(maps[("humidity", "location")]))
    part_2 = min(part_2, min(map(lambda r: r.start, location)))

In [10]:
print_part_1(part_1)
print_part_2(part_2)

Part 1: 282277027

Part 2: 11554135